# 🏥 Insurance prices 💰

Read more about dataset from [this kaggle page](https://www.kaggle.com/mirichoi0218/insurance).  The original source of the data is not mentioned in the description :(

We'll be predicting charges based on some features of the clients.

## General EDA and Viz

In [ ]:
%load_ext nb_black

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

data_url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"
insurance = pd.read_csv(data_url)

Do some general eda to get familiar with what data we have.

We can start our visualization journey with a `scatter_matrix` or an `sns.pairplot` to see all our continuous variables.  Categorical variables we might plot individually with our target since we have a managable number of columns.

* Create a `scatter_matrix` or an `sns.pairplot` of the data
* Note any correlations, patterns, and distributions you see

There's an interesting pattern in the scatterplot between `'age'` and `'charges'`.  It looks like there's 3 distinct groups.  This is hinting that there might be some variable interacting with `'age'`.  That is, the effect of `'age'` on `'charges'` might change based on some other variable at play.

Hopefully we can find this interacting variable in our data.  However, it might be explained by a variable outside of our dataset; some domain knowledge would then be very useful to try and bring what explains these gaps.

Let's visualize our categorical variables and try to figure out what might explain this.

* Create a for loop that iterates over all of the categorical columns
* Create a violin plot for each column with our `'charges'` column
* Interpret what you see

In [ ]:
cat_cols = ['smoker', 'sex', 'region']



* Smokers definitely appear to be charged more than non-smokers
* The distributions between the sexes are pretty similiar.  The main difference is that the male distribution has a 'fatter' right hand tail (i.e. a group of males getting charged more)
* Region seems to have a very small effect on charges.  Perhaps the southeast has a fatter tail than the others, but this may be less due to region itself, and more explained by other variables.  For example, I think I've read that the southeast has some of the highest rates of obesitity, idk if this is still accurate (or if it ever was).

We just saw some effects that might help us explain the weird patterns shown in the `'age'` x `'charges'` scatter plot.  Let's try to include smoker in a scatterplot with these 2 variables since it appears to have the most extreme effect.

* Create a `scatterplot` of `'age'` and `'charges'` colored by the `'smoker'` column
* What do you notice?

It looks like being a smoker interacts with age to explain part of this pattern of groups.  The effect of age on charges changes depending on whether or not the individual is a smoker.

This plot shows that being a smoker could explain the top and bottom groups, but what about the middle group?  It looks like there's still a pattern to be found.

The remaining bit of the pattern is a little bit harder to discover.  We're already showing 3 dimensions in our 2d plot thanks to color.  We could include a sizing dimension to try and bring in a 4th dimension, but this is harder to interpret.

* Add the `'bmi'` column to the scatter plot using the `size` argument of `sns.scatterplot`

Putting `'bmi'` looks to explain away some of the pattern as well. It's less obvious, but it looks like lower bmi smokers are in the middle group, while higher bmi smokers are in the top group.  There are exceptions to this though.  If I was a data scientist working on this problem I would want to discuss this plot with my clients to make sure I have all the features that account for these 3 'strips' of data.

## Model Prep

We'll keep all our features for now.  We might consider dropping region since it seems to have little effect based on our plots, but we would want to consult with experts (or build model with and without it and see if it has a positive effect on the accuracy or if its just dead weight).

In [ ]:
insurance.head()

In [ ]:
cat_cols = ["sex", "smoker", "region"]
# Dropping southeast dummy since its the most occuring category
# So from that I'm assuming it might be a good 'default'
drop_cats = ["male", "no", "southeast"]

num_cols = ["age", "bmi", "children"]

In [ ]:
X = insurance.drop(columns="charges")
y = insurance["charges"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

* Finish the `ColumnTransformer()` definition by adding a `StandardScaler()` that will be applied to the columns listed in the `num_cols` list

In [ ]:
preprocessing = ColumnTransformer(
    [
        ("encode_cats", OneHotEncoder(drop=drop_cats), cat_cols),
        ______
    ],
    remainder="passthrough",
)

* Finish the `PipeLine()` definition by adding an `SVR()` step

In [ ]:
# fmt: off
pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        _______
    ],
)
# fmt: on

* Fit the pipeline to the training data
* Score the model on train and testing data
    * What is this score?
    * Do we want to minimize it or maximize it?

In [ ]:
____

train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"train_score: {train_score}")
print(f"test_score: {test_score}")

Dang, that's horrible... Maybe we want a different `kernel`, maybe we want a different `C`, a different `epsilon`. Sounds easier to `GridSearchCV` for this than to do a lot of guess and check.

* Define a dictionary of parameters to be our grid to search through
    * Use `0.1`, `1`, & `10` for the potential values of `'epsilon'`
      * To specify parameters in a pipeline we have to provide the name of the step they apply to.  We specify the name of the step and then 2 underscores before using the usual hyperparameter name.  For example, if you named the `SVR()` step in your `Pipeline` `'svm'`, then the parameter name for `'epsilon'` would be `'svm__epsilon'`.
    * Try some of the kernels
    * Try a couple different degree polynomials
    * Note how many models you'll be building before running the grid search
* Define the grid search by passing in the pipeline, parameter grid
* Fit the grid search.
* Print out the best parameters.
    * Remember that `degree` is ignored by all `kernel`s except `'poly'`.  So if the best kernel isn't `'poly'` we can ignore the `degree` parameter chosen.

In [ ]:
grid = {
    "svm__epsilon": [0.1, 1, 10],
    ____,
    ____,
    ____,
}

pipeline_cv = GridSearchCV(____, ____, verbose=1)
pipeline_cv.fit(X_train, y_train)

print('Best hyperparameters:')
pipeline_cv._____

* Score the resulting model that was fit via grid search cross validation
* If the fit is poor, take note of what hyperparameters were chosen, are any of them at the edges of the grid?  Adjusting the grid might improve fit.

In [ ]:
train_score = pipeline_cv.score(X_train, y_train)
test_score = pipeline_cv.score(X_test, y_test)

print(f"train_score: {train_score}")
print(f"test_score: {test_score}")

* Make predictions on the test set
* Calculate the mean absolute error (MAE) and the mean absolute percent error (MAPE)
* Create a plot of the fitted vs the actual values